In [1]:
import torch
import numpy as np
import cv2
from models.enet import ENet
import os
from os.path import join

import torch
import torch.nn as nn
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
import torch.utils.data as data
import torchvision.transforms as transforms

from PIL import Image

import transforms as ext_transforms
from models.enet import ENet
from train import Train
from test import Test
from metric.iou import IoU
from args import get_arguments
from data.utils import enet_weighing, median_freq_balancing
import utils

In [24]:
path = '/home/jinwei/Documents/Git/PyTorch-ENet/park_save3.0/park_model'

In [25]:
ckp = torch.load(path)

In [26]:
ckp.keys()

dict_keys(['epoch', 'miou', 'state_dict', 'optimizer'])

In [27]:
torch.save(ckp['state_dict'], "/home/jinwei/segmodel3.0")

In [2]:
device = torch.device('cuda')
model = ENet(2).to(device)
model.eval()

ENet(
  (initial_block): InitialBlock(
    (main_branch): Conv2d(3, 13, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (ext_branch): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (batch_norm): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (out_activation): PReLU(num_parameters=1)
  )
  (downsample1_0): DownsamplingBottleneck(
    (main_max1): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (ext_conv1): Sequential(
      (0): Conv2d(16, 4, kernel_size=(2, 2), stride=(2, 2), bias=False)
      (1): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): PReLU(num_parameters=1)
    )
    (ext_conv2): Sequential(
      (0): Conv2d(4, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): PReLU(num_parameters=1)
    )
    (ext_c

In [34]:
model.load_state_dict(torch.load("/home/jinwei/segmodel3.0"))

In [59]:
input_path = '/home/jinwei/Documents/Git/PyTorch-ENet/data/aldrichpark/val/1565029757550.png'
input_path = '/home/jinwei/Downloads/road1.jpg'
input_path = '/home/jinwei/Documents/Git/SfmLearner-Pytorch/data/sfmtest/test4.jpg'
#input_path = '/home/jinwei/Downloads/8.6_2pm/pics/1565116884341.jpg'
inputs = (torch.from_numpy(np.transpose(Image.open(input_path), (2,0,1))).float()/255).to(device)

In [60]:
output = model(inputs.unsqueeze(0))

In [61]:
output.shape

torch.Size([1, 2, 240, 320])

In [62]:
output = output.detach().squeeze()

In [63]:
output = output.argmax(0)

In [64]:
semantic_img = Image.fromarray((output.cpu().numpy()*255).astype(np.uint8))

In [65]:
semantic_img.save("/home/jinwei/Documents/Git/SfmLearner-Pytorch/data/sfmtest/test4_road.png","PNG")

In [7]:
device = torch.device('cuda')

In [23]:
# extract images
direc_path = "/home/jinwei/Documents/Notebooks/images"
pics  = [f for f in os.listdir(direc_path) if ("_enet" not in f and "png" in f)]
for p in pics:
    inputs = (torch.from_numpy(np.transpose(Image.open(join(direc_path,p)), (2,0,1))).float()/255).to(device)
    road_img = Image.fromarray((model(inputs.unsqueeze(0)).detach().squeeze().argmax(0).cpu().numpy()*255).astype(np.uint8))
    road_img.save(join(direc_path, p.replace(".png", "_enet.png")))

In [ ]:
outputs = model(inputs)